In [56]:
# strongly inspired by https://towardsdatascience.com/analyzing-coronavirus-covid-19-data-using-pandas-and-plotly-2e34fe2c4edc
## Import Libraries
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression 
from sklearn.preprocessing import PolynomialFeatures 
plt.rcParams['figure.figsize'] = [15, 5]
from IPython import display
from ipywidgets import interact, widgets
import datetime

## Read Data for Cases, Deaths and Recoveries
srcconfirmed="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
srcrecovered="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
srcdeaths="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"

Cases_confirmed=pd.read_csv(srcconfirmed)
Cases_recovered=pd.read_csv(srcrecovered)
Cases_deaths=pd.read_csv(srcdeaths)

print("Data source: https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/")
print("Retrieved on: " + str(datetime.datetime.now()))

Data source: https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/
Retrieved on: 2020-04-06 17:49:49.457239


In [57]:
def fix_date(df):
    return datetime.datetime.strptime(df["date"], '%m/%d/%y').strftime('%Y-%m-%d')

def fix_dataset(cases, value_name):
    cases = cases.rename(columns={'Province/State':'label', 'Country/Region':'parent'})
    cases.drop(['Lat', 'Long'], axis=1, inplace=True)
    cases = cases.melt(id_vars=['label','parent'], var_name="date", value_name=value_name)
    cases.fillna(value="", inplace=True)
    cases["date"] = cases.apply(fix_date, axis=1)
    cases.set_index(["parent","label","date"], inplace=True)
    return cases

# Transform
Cases_confirmed_clean = fix_dataset(Cases_confirmed, "confirmed")
Cases_recovered_clean = fix_dataset(Cases_recovered, "recovered")
Cases_deaths_clean = fix_dataset(Cases_deaths, "deaths")

Cases_raw = Cases_deaths_clean.merge(Cases_confirmed_clean, left_on=['parent','label','date'], right_on=['parent','label','date'])
Cases_raw = Cases_raw.merge(Cases_recovered_clean, left_on=['parent','label','date'], right_on=['parent','label','date'])
Cases_raw.loc['US']

deaths  confirmed  recovered
label date                                    
      2020-01-22       0          1          0
      2020-01-23       0          1          0
      2020-01-24       0          2          0
      2020-01-25       0          2          0
      2020-01-26       0          5          0
...                  ...        ...        ...
      2020-04-01    4757     213372       8474
      2020-04-02    5926     243453       9001
      2020-04-03    7087     275586       9707
      2020-04-04    8407     308850      14652
      2020-04-05    9619     337072      17448

[75 rows x 3 columns]

In [58]:
def add_derived(Cases):
#    Cases.set_index('date', inplace=True)
    Cases['active'] = Cases['confirmed']-Cases['deaths']-Cases['recovered']
    Cases['lethality'] = np.round(Cases['deaths']/Cases['confirmed'], 3)
    
#    Cases.drop(['derivedConfirmed_'], axis=1, inplace=True)
    return Cases

# Get Daily Data
Cases_diff = Cases_raw.groupby(['parent','label','date'])
Cases_diff = Cases_diff.sum()
Cases_diff = Cases_diff.diff().fillna(0)
Cases_diff = Cases_diff.rename(columns={"confirmed":"confirmed_new","deaths":"deaths_new","recovered":"recovered_new"})
Cases = Cases_raw.merge(Cases_diff, left_on=['parent','label','date'], right_on=['parent','label','date'])
Cases = add_derived(Cases)

Cases.to_csv('out_world.csv')
Cases.loc['Germany'].to_csv('out_DE.csv')
Cases.loc['Germany']
#Cases

deaths  confirmed  recovered  deaths_new  confirmed_new  \
label date                                                                  
      2020-01-22       0          0          0        -2.0         -174.0   
      2020-01-23       0          0          0         0.0            0.0   
      2020-01-24       0          0          0         0.0            0.0   
      2020-01-25       0          0          0         0.0            0.0   
      2020-01-26       0          0          0         0.0            0.0   
...                  ...        ...        ...         ...            ...   
      2020-04-01     920      77872      18700       145.0         6064.0   
      2020-04-02    1107      84794      22440       187.0         6922.0   
      2020-04-03    1275      91159      24575       168.0         6365.0   
      2020-04-04    1444      96092      26400       169.0         4933.0   
      2020-04-05    1584     100123      28700       140.0         4031.0   

                  recovered_new  active  lethality  
label date                                          
      2020-01-22          -36.0       0        NaN  
      2020-01-23            0.0       0        NaN  
      2020-01-24            0.0       0        NaN  
      2020-01-25            0.0       0        NaN  
      2020-01-26            0.0       0        NaN  
...                         ...     ...        ...  
      2020-04-01         2600.0   58252      0.012  
      2020-04-02         3740.0   61247      0.013  
      2020-04-03         2135.0   65309      0.014  
      2020-04-04         1825.0   68248      0.015  
      2020-04-05         2300.0   69839      0.016  

[75 rows x 8 columns]

In [209]:
# Shift data
#CasesShiftedB = Cases.tail(0)
#for country in Cases.droplevel('date').index.unique().tolist():
#    firstcase = Cases.loc[country]['deaths'].reset_index().set_index('date')
#    firstcase = firstcase[firstcase.ne(0)].dropna().reset_index()
#    firstcase['parent'] = country
#    firstcase = firstcase.set_index(['parent', 'label', 'date'])
#    CasesShiftedB = CasesShiftedB.append(firstcase)
#CasesShiftedB

#CasesS = {}

#for place in Cases.droplevel('date').index.unique().tolist():
#    firstcase = Cases.loc[place]['deaths'].reset_index().set_index('date')
#    firstcase = firstcase[firstcase.ne(0)]
#    CasesS[place] = firstcase

#CasesS

def add_day(df):
    firstcase = Cases.loc[place]['deaths'].reset_index().set_index('date')
    Cases['day'] = Cases.loc[place][firstcase.ne(0)]
    return day

def combine_projectedDeaths(row):
    if row['deaths'] == False:
        return row['projectedDeaths']
    return row['deaths']

def add_simulated(Cases):
    Cases['deaths_bak'] = Cases['deaths']
    Cases['deaths'] = Cases.apply(combine_projectedDeaths, axis=1)
    Cases['derivedConfirmed_'] = np.round(Cases['deaths'] / 0.03, 0) # mortality
    Cases['derivedConfirmed'] = Cases['derivedConfirmed_'].shift(periods=-14) # time to death after turning infectious
    
    Cases['derivedRecovered_'] = Cases['derivedConfirmed_'].shift(periods=-3) # periods = time to death + time to recovery
    Cases['derivedRecovered'] = Cases['derivedRecovered_'] - Cases['deaths']
    
    Cases['derivedActive'] = Cases['derivedConfirmed_'].shift(periods=-12) - Cases['deaths'] - Cases['derivedRecovered'] # periods = time to death + incubation period
    Cases['derivedLethality'] = np.round(Cases['deaths']/Cases['derivedConfirmed'], 3)
    Cases['testRate'] = np.round(Cases['confirmed']/Cases['derivedConfirmed'], 3)
    
    Cases['deaths'] = Cases['deaths_bak']
    Cases.drop(['derivedConfirmed_','derivedRecovered_','deaths_bak'], axis=1, inplace=True)
    return Cases

#Cases_sim = add_simulated(Cases.loc['Germany'].droplevel('label'))
firstrecordeddate = Cases.reset_index().head(1)['date'].values[0]
lastrecordeddate = Cases.reset_index().tail(1)['date'].values[0]
startdate = datetime.datetime.strptime(lastrecordeddate, '%Y-%m-%d')
enddate = (startdate + datetime.timedelta(days=(120 - 15))).strftime('%Y-%m-%d')

def extenddata(cases, title=""):
    y = cases['deaths'].values
    x = range(0, y.size)
    cases = cases.reset_index().set_index('date')

    # calculate polynomial
    z = np.polyfit(x[-7:], y[-7:], 2)
#    z = np.polyfit(x, y, 2)
    f = np.poly1d(z)
    print(f)

    # calculate new x's and y's
    lookahead = 120
    cutoffsimulated = -15 # days we calculate in the simulation, but later don't display
    cutoffpolynomial = -14 # days we calculate in the polynomial, but later don't display
    x_new = np.linspace(x[-1], x[-1]+lookahead, lookahead)
    y_new = f(x_new)
    
    # prepare new date range
    startdate = datetime.datetime.strptime(cases.index[-1], '%Y-%m-%d')
    x_new_dates = [startdate] # in the fit graph we need to start one day early for it to look smooth
    for x_delta in range(1,lookahead): # generate future dates
        date = (startdate + datetime.timedelta(days=x_delta)).strftime('%Y-%m-%d')
        x_new_dates.append(date)
        cases = cases.append(pd.DataFrame({'date':[date], 'deaths': False, 'projectedDeaths':[np.round(f(x[-1]+1+x_delta),0)]}).set_index("date"))
#        print('delta '+date+" y "+str(f(x[-1]+1+x_delta)))

    # cut displayed data and add simulated numbers
    firstdate = cases.index[1]
    enddate = (startdate + datetime.timedelta(days=(lookahead + cutoffsimulated))).strftime('%Y-%m-%d')
#    date = (startdate + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
#    cases = add_simulated(cases)[date:enddate].fillna(0)
    cases = add_simulated(cases)[firstdate:enddate].fillna(0)
#    y_new = y_new[:cutoffpolynomial]

    # export data
    if title != "":
        cases.to_csv("sim_"+title+".csv")
        cases.to_html("sim_"+title+".html")
#        cases.to_excel("output.xslx")

    # in the fit graph we need to start one day early for it to look smooth
    cases = cases.append(pd.DataFrame({'date':[startdate], 'deaths':[f(x[-1]+1)]}).set_index("date"))
        
    return cases

GlobalTotals = Cases.reset_index().groupby('date').sum()
#GlobalTotals = extenddata(GlobalTotals)
#GlobalTotals.loc["2020-03-01":"2020-03-14"]
#GlobalTotals
extenddata(Cases.loc['Germany'])
#Cases.loc['Germany']

       2
1.571 x - 62.07 x - 2414


,label,deaths,confirmed,recovered,deaths_new,confirmed_new,recovered_new,active,lethality,projectedDeaths,derivedConfirmed,derivedRecovered,derivedActive,derivedLethality,testRate
date,,,,,,,,,,,,,,,
2020-01-23,,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0
2020-01-24,,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0
2020-01-25,,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0
2020-01-26,,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0
2020-01-27,,0.000000,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-07-16,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,35831.0,1435267.0,1208436.0,155333.0,0.025,0.0
2020-07-17,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36327.0,1453267.0,1224773.0,156267.0,0.025,0.0
2020-07-18,0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,36826.0,1471367.0,1241207.0,157234.0,0.025,0.0


In [262]:
%matplotlib inline
plt.style.use('seaborn-dark')

Cases = Cases.reset_index().set_index(['parent','label'])

def plotData(cases, title):
    cases = cases.reset_index().set_index('date')
    sim = extenddata(cases, title)
#    sim = extenddata(cases[firstrecordeddate:"2020-04-04"], title)
    projected = sim[lastrecordeddate:]
    
    fig = make_subplots(rows=3, cols=2,shared_xaxes=True,
                        specs=[[{}, {}],[{},{}],
                           [{"colspan": 2}, None]],
                        subplot_titles=('Total Confirmed Cases','Active Cases','Deaths','Recoveries','Death to Cases Ratio'))
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=cases.index,y=cases['confirmed'],
                             mode='lines',
                             name='Confirmed Cases',
                             line=dict(color='red',width=2)))
#                             row=1,col=1)
    fig.add_trace(go.Scatter(x=cases.index,y=cases['active'],
                             mode='lines',
                             name='Active Cases',
                             line=dict(color='purple',width=2)))
#                             row=1,col=2)
    fig.add_trace(go.Scatter(x=cases.index,y=cases['recovered'],
                             mode='lines',
                             name='Recoveries',
                             line=dict(color='green',width=2)))
#                             row=1,col=1)
    fig.add_trace(go.Scatter(x=cases.index,y=cases['deaths'],
                             mode='lines',
                             name='Deaths',
                             line=dict(color='black',width=2)))
#                             row=1,col=1)
    fig.add_trace(go.Scatter(x=projected.index,y=projected['projectedDeaths'],
                             mode='lines',
                             name='Projected deaths',
                             line=dict(color='black',width=2,dash='dot')))
#                             row=1,col=1)
        
    fig.add_trace(go.Scatter(x=sim.index,y=sim['derivedConfirmed'],
                             mode='lines',
                             name='Derived Cases',
                             line=dict(color='red',width=2,dash='dot')))
                             #row=1,col=1)
    fig.add_trace(go.Scatter(x=sim.index,y=sim['derivedActive'],
                             mode='lines',
                             name='Derived Active Cases',
                             line=dict(color='purple',width=2,dash='dot')))
                             #row=1,col=2)
    fig.add_trace(go.Scatter(x=sim.index,y=sim['derivedRecovered'],
                             mode='lines',
                             name='Derived Recoveries',
                             line=dict(color='green',width=2,dash='dot')))
                             #row=2,col=2)

    #fig.update_layout(showlegend=False)
    fig.update_layout(title=title,
                       yaxis_title='Cases',
#                       yaxis_title='Cases (log)', yaxis_type="log",
                       xaxis_title='Date')

    fig.show()
#plotData(add_simulated(GlobalTotals), 'Covid-19 worldwide')
plotData(GlobalTotals, 'Global')
plotData(Cases.loc["US"], 'US')
#plotData(Cases.loc["Germany"].droplevel("label").loc["2020-03-14":"2020-04-04"], 'Germany')
plotData(Cases.loc['Germany'], 'Germany')
#plotData(Cases.loc["China"].groupby('date').sum(), 'China')

       2
68.87 x - 4368 x + 1.573e+04


       2
44.63 x - 5219 x + 1.515e+05


       2
1.571 x - 62.07 x - 2414


In [49]:
Cases.loc['Germany']

,date,deaths,confirmed,recovered,deaths_new,confirmed_new,recovered_new,active,lethality,deaths.bak
label,,,,,,,,,,
,2020-01-22,0,0,0,-2.0,-174.0,-36.0,0,NaN,0
,2020-01-23,0,0,0,0.0,0.0,0.0,0,NaN,0
,2020-01-24,0,0,0,0.0,0.0,0.0,0,NaN,0
,2020-01-25,0,0,0,0.0,0.0,0.0,0,NaN,0
,2020-01-26,0,0,0,0.0,0.0,0.0,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...
,2020-04-01,920,77872,18700,145.0,6064.0,2600.0,58252,0.012,920
,2020-04-02,1107,84794,22440,187.0,6922.0,3740.0,61247,0.013,1107
,2020-04-03,1275,91159,24575,168.0,6365.0,2135.0,65309,0.014,1275


In [261]:
totalcases = Cases.max(level=0)['deaths'].reset_index().set_index('parent')
totalcases = totalcases.sort_values(by='deaths',ascending=False)
top10 = totalcases.head(10)
fig = go.Figure(go.Bar(x=top10.index, y=top10['deaths'],
                      text=top10['deaths'],
            textposition='outside'))
fig.update_layout(title_text='Top 10 Countries by Deaths')
fig.update_yaxes(showticklabels=False)

fig.show()

In [254]:
countries = ["France","Italy","Spain","US","United Kingdom","Iran","China","Netherlands","Germany","Belgium","Sweden"]
sims_lastrecordeddate = {}
sims_enddate = {}
columns = {}
for place in countries:
    print(place)
    sim = extenddata(Cases.loc[place].reset_index().set_index('date'), place)
    sims_lastrecordeddate[place] = sim.loc[lastrecordeddate]
    sims_enddate[place] = sim.loc[enddate]
    if "label" not in sims_lastrecordeddate[place].index.values:
        sims_lastrecordeddate[place] = sims_lastrecordeddate[place].reset_index().groupby("date").sum().loc[lastrecordeddate]
        sims_enddate[place] = sims_enddate[place].drop("label")
    else:
        sims_lastrecordeddate[place] = sims_lastrecordeddate[place].drop("label")
        sims_enddate[place] = sims_enddate[place].drop("label")
    columns = sims_lastrecordeddate[place].index
    sims_lastrecordeddate[place] = sims_lastrecordeddate[place].values
    sims_enddate[place] = sims_enddate[place].values
    print("ok")
    
pd_sims = pd.DataFrame(sims_lastrecordeddate, columns = sims_lastrecordeddate.keys())
pd_sims.insert(0, 'name', columns)
pd_sims.set_index('name', inplace=True)
pd_sims_enddate = pd.DataFrame(sims_enddate, columns = sims_enddate.keys())
pd_sims_enddate.insert(0, 'name', columns)
pd_sims_enddate.set_index('name', inplace=True)

France
         2
-0.2381 x + 968.1 x - 6.332e+05
ok
Italy
        2
-21.17 x + 3730 x - 1.442e+05
ok
Spain
        2
-16.21 x + 3144 x - 1.311e+05
ok
US
       2
44.63 x - 5219 x + 1.515e+05
ok
United Kingdom
        2
-176.2 x + 2.891e+05 x - 1.186e+08
ok
Iran
       2
2.024 x - 148 x + 3466
ok
China
         2
-0.1548 x + 764.9 x - 9.45e+05
ok
Netherlands
       2
98.08 x - 7.283e+04 x + 1.352e+07
ok
Germany
       2
1.571 x - 62.07 x - 2414
ok
Belgium
        2
-1.869 x + 418 x - 1.926e+04
ok
Sweden
        2
-3.429 x + 532.2 x - 2.02e+04
ok


In [256]:
#pd_sims_enddate
pd_sims

,France,Italy,Spain,US,United Kingdom,Iran,China,Netherlands,Germany,Belgium,Sweden
name,,,,,,,,,,,
deaths,8093.000,15887,12641,9619,4.943000e+03,3603,3333.000,1771.000,1584,1447,401
confirmed,93773.000,128948,131646,337072,4.843600e+04,58226,82602.000,17953.000,100123,19691,6830
recovered,16349.000,21815,38080,17448,2.290000e+02,19736,77207.000,257.000,28700,3751,205
deaths_new,519.000,525,694,1212,6.230000e+02,151,3.000,115.000,140,164,28
confirmed_new,2925.000,4316,5478,28222,5.959000e+03,2483,59.000,1226.000,4031,1260,387
recovered_new,777.000,819,3861,2796,1.400000e+01,0,261.000,-5.000,2300,504,0
active,69331.000,91246,80925,310005,4.326400e+04,34887,2062.000,15925.000,69839,14493,6224
lethality,0.242,0.123,0.096,0.029,3.590000e-01,0.062,0.365,0.350,0.016,0.073,0.059
projectedDeaths,0.000,0,0,0,0.000000e+00,0,0.000,0.000,0,0,0


In [259]:
totalcases = pd_sims.loc['testRate'].sort_values(ascending=False)
top10 = totalcases.head(10)
fig = go.Figure(go.Bar(x=top10.index, y=top10.values,
                      text=top10.values,
            textposition='outside'))
fig.update_layout(title_text='Top Countries by Test Rate '+lastrecordeddate)
fig.update_yaxes(showticklabels=False)

fig.show()

In [258]:
totalcases = pd_sims.loc['derivedConfirmed'].sort_values(ascending=False)
top10 = totalcases.head(10)
fig = go.Figure(go.Bar(x=top10.index, y=top10.values,
                      text=top10.values,
            textposition='outside'))
fig.update_layout(title_text='Top Countries by Derived Cases '+lastrecordeddate)
fig.update_yaxes(showticklabels=False)

fig.show()

In [257]:
totalcases = pd_sims_enddate.loc['projectedDeaths'].sort_values(ascending=False)
top10 = totalcases.head(10)
fig = go.Figure(go.Bar(x=top10.index, y=top10.values,
                      text=top10.values,
            textposition='outside'))
fig.update_layout(title_text='Top Countries by Projected Deaths by '+enddate)
fig.update_yaxes(showticklabels=False)

fig.show()